In [24]:
# %reset

# BMI Multinomial Example

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler  # Normalization
from sklearn.model_selection import train_test_split # train, test 데이터 분리
from sklearn.model_selection import KFold  # cross validation

# Raw Data Loading
df = pd.read_csv('./data/bmi.csv', skiprows=3)

# 결측치와 이상치 확인 및 처리 (결측치와 이상치는 없기때문에 이 부분은 넘어가요!)

# Data Split
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height','weight']],df['label'],test_size=0.3, random_state=0)
# 7:3 비율로 train과 test 데이터를 분리했어요!!
# x_data_test, t_data_test 두개는 맨끝에서 우리 모델의 최종 Accuracy를 측정할때 
# 한번 사용!

# Normalization 진행(Min-Max Scaling)
scaler = MinMaxScaler()
scaler.fit(x_data_train)  # 나중에 scaling을 하기 위한 정보를 scaler에게 세팅
x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train   # 에러를 방지하기 위해서 사용하지 않는 변수를 삭제
del x_data_test

####### Tensorflow 구현 #######

# 정답에 해당하는 t_data_train을 살펴보니 이게.. Multinomial 이예요!
# 당연히 One-Hot Encoding을 이용해서 데이터를 변환시켜야 해요!
# 0 => 1 0 0
# 1 => 0 1 0
# 2 => 0 0 1
# 3 => 0 0 0
# 형태로 바꿔줘야 해요!
# Numpy를 이용해서 로직처리로 바꾸는 방법이 있고
# Tensorflow를 이용해서 API를 이용해서 바꾸는 방법이 있어요!
# Tensorflow node를 실행하기 위해서 session이 필요

sess = tf.Session()

# 우리가 사용할 label(t_data)을 one hot encoding 행태로 변환!!
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=3))
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=3))

del t_data_train  # 이것도 오류를 방지하기 위해서 제거!
del t_data_test

# 지금까지 위에서 
# x_data_train_norm , t_data_train_onehot 만들었어요!
# training data set을 준비했어요!

# 데이터가 준비되었으니 이제 Tensorflow Graph를 그려보아요!

# Placeholder
X = tf.placeholder(shape=[None,2], dtype=tf.float32)
T = tf.placeholder(shape=[None,3], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([2,3]), name='weight')
b = tf.Variable(tf.random.normal([3]), name='bias')

# Hypothesis(Model)
logit = tf.matmul(X,W) + b
H = tf.nn.softmax(logit)  # tf.sigmoid()

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, 
                                                                 labels=T))

# train 
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

# 반복학습하는 함수
# parameter
num_of_epoch = 1000
batch_size = 100   # 한번에 학습할 x_data와 t_data의 행의 수

def run_train(sess,train_x, train_t):
    print('### 학습이 시작되요!! ###')
    sess.run(tf.global_variables_initializer())     
    total_batch = int(train_x.shape[0] / batch_size)
    
    for step in range(num_of_epoch):
        
        for i in range(total_batch):   
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train, loss],
                                   feed_dict={X:batch_x,
                                              T:batch_t})
        if step % 100 == 0:
            print('Loss : {}'.format(loss_val))
            
    print('### 학습이 종료되요!! ###')
    
# axis 값인데 2차원에서 0이면 행방향, 1이면 열방향.
# Accuracy(정확도측정)      #    0     1      2  
predict = tf.argmax(H,1)    # [[0.5   0.4   0.1]]   내가 입력으로 넣은 값에 대한 예측
correct = tf.equal(predict, tf.argmax(T,1)) 
accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))
   

# 학습을 진행하고 
# run_train(sess,x_data_train_norm,t_data_train_onehot)
# Accuracy 측정 (Training data로 validation을 수행해보아요!)
# result = sess.run(accuracy,feed_dict={X:x_data_train_norm,
#                                       T:t_data_train_onehot})
# print('Training data로 validation한 정확도 : {}'.format(result))       
# 이렇게 하는건 좋지 않아요!!
# 이렇게 하는것 보다는 Cross Validation을 하는게 좋아요!!


# Cross Validation
cv = 5        # [훈련, 검증] => 5 set가 만들어져요!
results = []  # 5 set에 대한 accuracy를 구해서 list안에 차곡차곡 넣을꺼예요
kf = KFold(n_splits=cv, shuffle=True)

for training_idx, validation_idx in kf.split(x_data_train_norm):
    # training_idx : 결국은 index값을 알아와요!!
#     train_x = x_data_train_norm[training_idx]   # Fancy indexing
    train_x = x_data_train_norm[training_idx]   # Fancy indexing
    train_t = t_data_train_onehot[training_idx]

    valid_x = x_data_train_norm[validation_idx]
    valid_t = t_data_train_onehot[validation_idx]
    
    run_train(sess,train_x,train_t)
    results.append(sess.run(accuracy, 
                            feed_dict={X:valid_x,
                                       T:valid_t}))
print('Cross Validation 결과 : {}'.format(results))    
print('Cross Validation 최종 결과 : {}'.format(np.mean(results)))   

# 학습진행    
run_train(sess,x_data_train_norm,t_data_train_onehot)
# Accuracy 측정
result = sess.run(accuracy, feed_dict={X:x_data_test_norm,
                                       T:t_data_test_onehot})
print('최종 정확도 : {}'.format(result))   



### 학습이 시작되요!! ###
Loss : 0.9132822155952454
Loss : 0.187893345952034
Loss : 0.13969911634922028
Loss : 0.11754029244184494
Loss : 0.10422807931900024
Loss : 0.09513935446739197
Loss : 0.08844202011823654
Loss : 0.08324732631444931
Loss : 0.07906737923622131
Loss : 0.07561101764440536
### 학습이 종료되요!! ###
### 학습이 시작되요!! ###
Loss : 0.7421250939369202
Loss : 0.20145593583583832
Loss : 0.15794749557971954
Loss : 0.13779966533184052
Loss : 0.12567757070064545
Loss : 0.1173987090587616
Loss : 0.11129676550626755
Loss : 0.10656329989433289
Loss : 0.10275445878505707
Loss : 0.09960313141345978
### 학습이 종료되요!! ###
### 학습이 시작되요!! ###
Loss : 0.974588930606842
Loss : 0.21209770441055298
Loss : 0.16459840536117554
Loss : 0.1429629623889923
Loss : 0.1300310492515564
Loss : 0.12122791260480881
Loss : 0.11475172638893127
Loss : 0.10973434150218964
Loss : 0.10570018738508224
Loss : 0.10236560553312302
### 학습이 종료되요!! ###
### 학습이 시작되요!! ###
Loss : 0.8933748006820679
Loss : 0.21547317504882812
Loss : 0.1691

In [29]:
# Prediction!!
height = 187
weight = 78

my_state = [[height,weight]]
my_state_scaled = scaler.transform(my_state)
print(my_state_scaled)

result = sess.run(H, feed_dict={X:my_state_scaled})
print(np.argmax(result))

[[0.8375     0.95555556]]
1
